# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [ ]:
import pandas as pd

## imports for Python, Pandas

In [ ]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [ ]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [ ]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

In [ ]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [ ]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

In [ ]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [ ]:
# Read in the file
df = pd.read_json('data/world_bank_projects.json')
df.head()

In [ ]:
# Countries with most projects
df['countryname'].value_counts().head(10)

In [ ]:
# Major project themes
# Approach: 
# 1. Extract Panda Series for mjtheme_namecode
# 2. Use json_normalize to build a dataframe with (code, name) columns for each row.
# 3. Use pd.concat to concatenate all dataframes.
# 4. Use value_counts to count the occurences for all codes.
pd.concat([json_normalize(text) for text in df['mjtheme_namecode']], ignore_index=True)['code'].value_counts().head(10)

In [ ]:
# Build code to names dictionary
# Approach:
# 1. Extract Panda Series for mjtheme_namecode
# 2. Use json_normalize to build a dataframe with (code, name) columns for each row.
# 3. Use pd.concat to concatenate all dataframes.
# 4. Filter all rows with empty strings
# 5. Drop Duplicates
# 6. Set Index as code column
# 7. Build a dictionary by iterating over data frame rows.
code_names = {k:v['name'] for k, v in pd.concat([json_normalize(text) for text in df['mjtheme_namecode']]).loc[lambda x: x.name != ''].drop_duplicates().set_index('code').iterrows()}
code_names

In [ ]:
# Populate new dataframe values from this dictionary
# 1. Create a copy of the data frame
# 2. Create a dataframe of code, name columns as before.
# 3. Create a dictionary entry by applying a lambda function row-wise to dataframe. The lambda function creates a dictionary with
#    code and name keys and name values are replaced by code_names dictionary values.
# 4. Create a Panda Series and apply the results back to the copy of the data frame created.
dt = df.copy()
dt['mjtheme_namecode'] = pd.Series(list(json_normalize(text).apply(lambda row: {'code': row['code'], 'name': code_names[row['code']]}, axis=1)) for text in df['mjtheme_namecode'])

In [ ]:
# Check if all values are populated
assert pd.concat([json_normalize(text) for text in dt['mjtheme_namecode']], ignore_index=True).loc[lambda x: x.name == ''].shape[0] == 0